In [1]:
from model import lstm, bidirectional_lstm, gru
from utils import read_raw_dataset, select_features_name, read_chunk, train_with_data
from fed_algo import fed_avg

ModuleNotFoundError: No module named 'keras'

In [ ]:
X_raw, y = read_raw_dataset(
    "datasets/pokemon.csv", index_col="id", target_name="MOS", drop=["user_id"]
)
features = select_features_name(X_raw, y)
X_test, y_test = read_chunk("pokemon", "-test", features)
X_init, y_init = read_chunk("pokemon", "-init", features)
result = {
    "num_clients": [25, 50, 75, 100, 125, 150],
    "lstm": [],
    "bidirectional": [],
    "gru": [],
}

In [ ]:
models = [lstm(), bidirectional_lstm(), gru()]
models_name = ["lstm", "bidirectional", "gru"]
for model in models:
    train_with_data(model, model.get_weights(), (X_init, y_init))

In [ ]:
num_clients = [25, 50, 75, 100, 125, 150]

for num_client in num_clients:
    datasets = [
        read_chunk("pokemon", chunk_id + 1 + num_client - 25, features)
        for chunk_id in range(25)
    ]

    for name, model in zip(models_name, models):
        old_weights = model.get_weights()
        local_models = [
            train_with_data(model, old_weights, dataset) for dataset in datasets
        ]
        new_weights = fed_avg(local_models)
        model.set_weights(new_weights)
        result[name].append(model.evaluate(X_test, y_test)[0])

In [ ]:
import json

json_result = json.dumps(result, indent=4)
print(json_result)